In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import torchaudio
import time
# to fix relative import problem
import os
import sys
sys.path.append('../')
import src.datasets.soccernet_ms_extracted as dataset

from torch.utils.data import Dataset, DataLoader

/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
root_dir   = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/"
train_file = root_dir+"listgame_Train_300.npy"
valid_file = root_dir+"listgame_Valid_100.npy"
test_file  = root_dir+"listgame_Test_100.npy"

In [3]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
%%bash
nvidia-smi

Thu Dec 10 14:10:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:E7:00.0 Off |                    0 |
| N/A   33C    P0    50W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#  Test loading of some samples from train_samples.npy

# Dataloader

In [6]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Note: Can also see warning once
# warnings.filterwarnings(action='once')

In [6]:
params = {'batch_size': 12,
         'shuffle': True,
         'num_workers':4}

In [ ]:
trainloader = DataLoader(X_train_npy,**params)
testloader = DataLoader(X_test_npy,**params)

In [8]:
X_train_npy[0]

{'gameTime': '1 - 13:10', 'label': 'soccer-ball', 'team': 'home', 'duration1': 'Duration: 00:45:00.00', 'duration2': 'Duration: 00:45:00.00'}


{'path': '/work/oarongve/data/sound_dataset/SoccerNet-code/data/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/0_ms.npy',
 'ms': array([[[ 0.3697104 , -1.9689162 , -4.3114905 , ..., -4.1874843 ,
          -4.4186516 , -4.0682044 ],
         [ 0.39576104, -1.942303  , -4.1987357 , ..., -4.090648  ,
          -4.33303   , -4.065091  ],
         [ 0.6711389 , -1.66066   , -3.4416826 , ..., -3.385782  ,
          -3.6704643 , -4.1973424 ],
         ...,
         [-0.8570114 , -0.695406  ,  0.23381191, ..., -2.7876408 ,
          -2.9752624 , -4.062506  ],
         [-1.6340435 , -0.92085844, -0.90301675, ..., -3.304437  ,
          -3.8753335 , -4.149126  ],
         [-1.1152141 , -2.0846589 , -1.8565004 , ..., -4.2993913 ,
          -5.5238667 , -4.6549234 ]]], dtype=float32),
 'idx': 0,
 'label': 2,
 'info': {'gameTime': '1 - 13:10',
  'label': 'soccer-ball',
  'team': 'home',
  'duration1': 'Duration: 00:45:00.00',
  'duration2': 'Duration: 00:45:00.00'}}

In [ ]:
std = list()
mean = list()

for e in X_train_npy:
    print(e['ms'].mean())
    mean.append(e['ms'].mean())
    std.append(e['ms'].std())

In [ ]:
c = 0
for s in range(len(X_train_npy)):
    if np.isnan(X_train_npy[s]['ms']).any():
        print("found one!")
        print(X_train_npy[s]['idx'])
        c+=1

In [ ]:
X_train_npy[5]

In [ ]:
np.isnan(np.array(mean).any())

In [ ]:
np.isnan(np.array(mean)[0:3000])

In [ ]:
np.isnan(np.array(mean)[2220:2230]).any()

In [ ]:
np.array(mean)[2220:2230]

# Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(2208, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 2208)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/simple_net3')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()

In [ ]:
net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.025, momentum=0.9)

In [ ]:
%%bash
nvidia-smi

In [ ]:

# Dataloader or dataset faulty(returns batches of 0
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['ms'].to(device),data['label'].to(device)

        #print(inputs.size())
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every 2000 mini-batches
            
            writer.add_scalar('training loss',
                            running_loss / 5,
                            epoch * len(trainloader) + i)
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0
    
    # calculate accuracy
    with torch.no_grad():
        net.eval()
        res = torch.zeros((3,3))
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data['ms'].to(device),data['label'].to(device)

            # forward + backward + optimize
            outputs = net(inputs)
            
            preds = torch.argmax(outputs,dim=1)
            
            for p,gt in zip(preds,labels):
                res[int(p),int(gt)] += 1
            

        

        N_total = res.sum()
        N_correct = res.diag().sum()
        
        acc = N_correct / N_total
        
        writer.add_scalar('training acc',
                    acc,
                    epoch)
        print(f"Epoch : {epoch}, Accuracy : {acc}")
        
print('Finished Training')

# densenet


In [9]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/dense_net_3')

In [10]:
densenet = torchvision.models.densenet161(pretrained=True)
densenet.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
densenet.classifier = nn.Linear(in_features=densenet.classifier.in_features, out_features=3,bias=True)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(), lr=0.001, momentum=0.9)
densenet.to(device)

NameError: name 'nn' is not defined

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
densenet.to(device)

In [ ]:

# Dataloader or dataset faulty(returns batches of 0
epochs = 15
for epoch in range(epochs):
    running_loss = 0.0
    
    densenet.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['ms'].to(device),data['label'].to(device)

        #print(inputs.size())
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = densenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 20 == 0:    # print every 2000 mini-batches
            
            writer.add_scalar('training loss',
                            running_loss / 20,
                            epoch * len(trainloader) + i)
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0
    
    # calculate accuracy
    with torch.no_grad():
        densenet.eval()
        res = torch.zeros((3,3))
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data['ms'].to(device),data['label'].to(device)

            # forward + backward + optimize
            outputs = densenet(inputs)
            
            preds = torch.argmax(outputs,dim=1)
            
            for p,gt in zip(preds,labels):
                res[int(p),int(gt)] += 1
            

        

        N_total = res.sum()
        N_correct = res.diag().sum()
        
        acc = N_correct / N_total
        
        writer.add_scalar('training acc',
                    acc,
                    epoch)
        print(f"Epoch : {epoch}, Accuracy : {acc}")
        
print('Finished Training')

In [ ]:
with torch.no_grad():
        densenet.train()
        res = torch.zeros((3,3))
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data['ms'].to(device),data['label'].to(device)

            # forward + backward + optimize
            outputs = densenet(inputs)
            
            preds = torch.argmax(outputs,dim=1)
            print(outputs)
            for p,gt in zip(preds,labels):
                res[int(p),int(gt)] += 1
            

        

        N_total = res.sum()
        N_correct = res.diag().sum()
        
        acc = N_correct / N_total
        
        writer.add_scalar('training acc',
                    acc,
                    epoch)
        print(f"Epoch : {epoch}, Accuracy : {acc}")

In [ ]:
res.diag().sum() / res.sum()

# Save validation list for dataset loading

In [ ]:
X = soccernet_ms_npy_DS(root_dir=root_dir,npy_file=train_file)

In [ ]:
np.save(file=root_dir+"valid_samples.npy",arr=samples,allow_pickle=True)

In [ ]:
s = np.load(root_dir+"valid_samples.npy",allow_pickle=True)

In [ ]:
len(s)

# Save test list for dataset loading

In [ ]:
X = SoccerNetDataset(root_dir=root_dir,npy_file=test_file)

In [ ]:
samples = list()
for e in X:
    samples.append(e)
    

In [ ]:
np.save(file=root_dir+"test_samples.npy",arr=samples,allow_pickle=True)

In [ ]:
s = np.load(root_dir+"test_samples.npy",allow_pickle=True)

In [ ]:
len(s)

# Simple network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(2208, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 2208)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [ ]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Note: Can also see warning once
# warnings.filterwarnings(action='once')

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/mel_spec_experiment_4')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

In [ ]:
outputs

In [ ]:
(labels == torch.argmax(outputs, dim=1)).sum() 

In [ ]:
labels

In [ ]:
torch.argmax(outputs, dim=1)

In [ ]:
len(dataloader)

In [ ]:
writer = SummaryWriter('runs/mel_spec_experiment_4')

In [ ]:
net.to(device)
# Dataloader or dataset faulty(returns batches of 0
epochs = 2
for epoch in range(epochs):
    running_loss = 0.0
    
    net.train()
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['mel_spectogram'].to(device),data['label'].to(device)

        #print(inputs.size())
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every 2000 mini-batches
            
            writer.add_scalar('training loss',
                            running_loss / 5,
                            epoch * len(dataloader) + i)
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0
    
    # calculate accuracy
    with torch.no_grad():
        net.eval()
        res = torch.zeros((3,3))
        for i, data in enumerate(dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data['mel_spectogram'].to(device),data['label'].to(device)

            # forward + backward + optimize
            outputs = net(inputs)
            
            preds = torch.argmax(outputs,dim=1)
            
            for p,gt in zip(preds,labels):
                res[int(p),int(gt)] += 1
            

        

        N_total = res.sum()
        N_correct = res.diag().sum()
        
        acc = N_total / N_correct
        
        writer.add_scalar('training acc',
                    acc,
                    epoch)
        print(f"Epoch : {epoch}, Accuracy : {acc}")
        
print('Finished Training')

In [ ]:
writer.close()

# Evaluate samples to look for cases where y, sr from librosa is inconsistent

In [ ]:
outputs = net(inputs)

In [ ]:
inputs.size()

In [ ]:
net.fc1.weight

# With GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
net.to(device)

running_loss = 0.0
for i, data in enumerate(dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data['mel_spectogram'].unsqueeze(0).to(device),torch.argmax(data['one_hot_label']).to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

In [ ]:
%%bash
nvidia-smi

# Video dataset

In [6]:
# Clip dataset
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
import json
import torchvision
import datetime
from subprocess import Popen, PIPE
import re

class SoccerNetDataset(Dataset):
    """Soccernet Dataset"""
    
    def __init__(self,npy_file,
                 root_dir,nframes=1,
                 stride_frames=1,
                 frame_center='center',
                 transform=None,
                 train=False,
                 tshift={'active':False,
                         'mu':0,'sigma':(0.4/3), 
                         'interval':[-0.45,0.45],
                         'mode': 'uniform'}):
        self.npy_file = np.load(npy_file)
        self.samples = list() # maybe change structure later depending on efficiency        
        self.tshift = tshift
        self.root_dir = root_dir
        self.transform = transform
        self.frame_center = frame_center
        self.nframes = nframes
        self.stride_frames = stride_frames
        self.train = train
        # For each path in npy_file, get all annotations


        for e in self.npy_file:
            path, annotations = self.get_annotations(e)

            duration1 = self.getVideoLength(self.root_dir + e + "/1.mkv")
            duration2 = self.getVideoLength(self.root_dir + e + "/2.mkv")
            #print(f"duration1 : {duration1}, duration2: {duration2}")
            for annotation in annotations:
                # Check that annotations hold correct labels
                        if ("card" in annotation["label"]) or ("subs" in annotation["label"]) or ("soccer" in annotation["label"]):
                            annotation["duration1"] = duration1
                            annotation["duration2"] = duration2
                            self.samples.append([path,annotation,"soccernet"])


    def __len__(self):
        return len(self.samples)
    
    def getVideoLength(self,video_file):
        res = Popen(['ffmpeg', '-i', video_file, '-hide_banner'],stdout=PIPE,stderr=PIPE)
        none,meta = res.communicate()
        meta_out = meta.decode()
        #---| Take out info
        duration = re.search(r'Duration:.*', meta_out)
        return duration.group()[:21]

    def __getitem__(self,idx):
        """Returns a sample containing video path, clip and label"""
        if torch.is_tensor(idx):
            idx.tolist()
        
        if self.train:
            if idx in [2209,2210,2212,2213,2215,2217,2222]: # ultradirty hack - fix later
                idx = 0
        # get annotations
        time_half = int(self.samples[idx][1]["gameTime"][0])
        time_minute = int(self.samples[idx][1]["gameTime"][-5:-3])
        time_second = int(self.samples[idx][1]["gameTime"][-2:])
        annotation = self.samples[idx][1]
        source = self.samples[idx][2]
        


        # Get label
        if ("card" in annotation["label"]): label = 0
        elif ("subs" in annotation["label"]): label = 1
        elif ("soccer" in annotation["label"]): label = 2
        elif ("background" in annotation["label"]): label = 3
        else: 
            print("Warning, label not compatible with set")
            return
            
        # Get videopath
        if source == 'soccernet':
            vidpath = os.path.join(self.root_dir,
                                str(self.samples[idx][0]),
                                    str(time_half)+".mkv")
        else:
            vidpath = str(self.samples[idx][0])
        
        
        # Get video frames 
        
        # get start in second, use labeled time as center TODO: fix centerframe as keyframe and stride
        fps = 25.0 # assume fps = 25 for now, should be so
        start_sec = time_minute*60 + time_second
        end_sec = start_sec

        if self.nframes == 1:
            end_sec = start_sec
        
        if start_sec == 0:
            end_sec += (1/fps) # possibly unstable solution
            
        
        if self.frame_center == 'center' and self.nframes > 1:
            
            end_sec = end_sec + (self.nframes/fps) # might need to subtract 1/fps
            # Shift backwards to center around time but check that time > 0
            diff = (end_sec - start_sec) / 2 # TODO : Might result in bad precision
            temp_start_sec = start_sec - diff
            temp_end_sec = end_sec - diff
            
            # Only change as long as the shift operation doesnt shift out of bounds 
            if temp_start_sec >= 0:
                start_sec = temp_start_sec
                end_sec = temp_end_sec
                
            # TODO : Find new samplesize if self.stride_frames > 1
            # if self.stride_frames > 1:
            # For now, this is an operation for another place
                
        elif self.frame_center == 'back' and self.nframes > 1:
            print("This option should NOT be used during inference, please use 'center' instead")
            end_sec = end_sec + (self.nframes/fps) # might need to subtract 1/fps
        elif self.frame_center == 'front' and self.nframes > 1:
            #print("This option should NOT be used during inference, please use 'center' instead")
            
            end_sec = end_sec + (self.nframes/fps) # might need to subtract 1/fps
            
            # Shift forward such that the last frame is at annotated time t around time but check that time > 0
            diff = (end_sec - start_sec) # TODO : Might result in bad precision
            temp_start_sec = start_sec - diff
            temp_end_sec = end_sec - diff
            
            # Only change as long as the shift operation doesnt shift out of bounds 
            if temp_start_sec >= 0:
                start_sec = temp_start_sec
                end_sec = temp_end_sec

        
        # Temporal translation transform
        
        if self.tshift['active'] and self.frame_center == 'center' and source == 'soccernet':
            t0 = self.tshift['interval'][0]
            t = self.tshift['interval'][1]
            
            if self.tshift['mode'] == 'uniform':    
                delta = np.floor(np.random.uniform(t0,t) * self.nframes)
            elif self.tshift['mode'] == 'normal':
                mu = self.tshift['mu']
                sigma = self.tshift['sigma']
                temporal_window_size = self.nframes / 25.0
                delta = np.random.normal(mu,sigma)
                if delta < t0:
                    delta = t0
                elif delta > t:
                    delta = t
                delta = delta * temporal_window_size
            else: return "Please choose uniform or normal distribution"
            
            # change delta from frames to seconds with correct stepsize
            shifted_start = start_sec+delta
            shifted_end = end_sec+delta
            # Verify that shifted window stays inside 
            # get duration of video
            if time_half == 1:
                video_length = self.samples[idx][1]["duration1"]
            elif time_half == 2:
                video_length = self.samples[idx][1]["duration2"]

            video_length_min = video_length[-8:-6]
            video_length_sec = video_length[-5:-3]
            total_sec = int(video_length_min)*60 + int(video_length_sec)
            
            if shifted_start < 0 or shifted_end > total_sec:
                shifted_start = start_sec
                shifted_end = end_sec
            
            start_sec = shifted_start
            end_sec = shifted_end

        # Buffer to endsec incase of bad load
        end_sec = end_sec + 0.9 # loads more frames than needed, then reduced later
        clip,_,info = torchvision.io.read_video(vidpath, start_pts=start_sec, end_pts=end_sec, pts_unit='sec')
        
        # TODO : This should be tested
        clip = clip[:self.nframes,:,:,:]
        

        one_hot_labels = np.zeros(3)
        one_hot_labels[label] = 1

        csize = clip.size()
        abnormal_count = 0
        bad_count = 0   
        abnormal_2_count = 0
        # At this point clip is [T x H x W x C]
        if clip.size()[3] != 3 or clip.size()[0] != self.nframes or clip.size()[1] != 224 or clip.size()[2] != 398:
            abnormal_count += 1
            if clip.size()[1] != 224 or clip.size()[2] != 398 or clip.size()[0] / self.nframes < 0.6:
                clip = torch.zeros([self.nframes,224,398,3]).byte()
                bad_count += 1
            elif clip.size()[1] == 224 and clip.size()[2] == 398 and clip.size()[0] / self.nframes > 0.6:
                a = clip.size()[0]
                b = self.nframes - a
                last_f = clip[(a-1):,:,:,:]
                dup = last_f.repeat(b,1,1,1)
                clip = torch.cat((clip,dup)).byte()
                abnormal_2_count += 1

        

        sample = {'vidpath': vidpath,'clip': clip, 'annotation':annotation,'label':one_hot_labels,'idx':idx,
                 'csize':csize, 'source': source}
        

        if self.transform:
            sample['clip'] = self.transform(sample['clip'])
        return sample
            
    def get_annotations(self,path):
        """ Reads json files and returns """
        with open(self.root_dir+path+"/Labels.json") as jsonfile:
            json_label = json.load(jsonfile)
        
        labels = [e for e in json_label['annotations']]
        
        return path,labels
    def get_keyframe(self,idx):
        if self.frame_center == 'back': return self.__getitem__(idx)['clip'][0,:,:,:]
        elif self.frame_center == 'center': return self.__getitem__(idx)['clip'][self.nframes//2,:,:,:]
        elif self.frame_center == 'front': return self.__getitem__(idx)['clip'][self.nframes-1,:,:,:]
    def describe(self):
        card = 0
        subs = 0
        goal = 0
        background = 0

        for sample in self.samples:
            annotation = sample[1]
        # Get label
            if ("card" in annotation["label"]): card += 1
            elif ("subs" in annotation["label"]): subs +=1
            elif ("soccer" in annotation["label"]): goal += 1
            elif ("background" in annotation["label"]): background += 1

        print("Description of dataset\n\n")
        print("\n ********* Classes *********")
        print("\n card = 0\n subs = 1\n goals = 2\n background = 3")

        print("\n ********* Distribution and count *********")
        print(f"\n N card: {card} \n N subs: {subs} \n N goal: {goal} \n N background: {background} \n \n Total : {card+subs+goal+background}")
        
        print("\n\n ********* Configuration *********")
        print(f"\n npy_file: {self.npy_file} \n tshift: {self.tshift} \n root_dir: {self.root_dir} \n transform: {self.transform} \n frame_center: {self.frame_center} \n nframes: {self.nframes} \n stride_frames: {self.stride_frames} \n background: {self.background}")
        print("\n\n ********* End of description *********")

In [3]:
npy_file_train = "/work/oarongve/data/listgame_Train_300.npy"
npy_file_test = "/work/oarongve/data/listgame_Test_100.npy"
root_dir = "/work/oarongve/data/"

In [7]:
videods = SoccerNetDataset(root_dir=root_dir,npy_file=npy_file,train=True)

In [8]:
videods[0]

{'vidpath': '/work/oarongve/data/england_epl/2014-2015/2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal/1.mkv',
 'clip': tensor([[[[ 90,  71,  75],
           [ 60,  41,  45],
           [ 72,  50,  53],
           ...,
           [ 71,  61,  65],
           [125, 115, 119],
           [109,  99, 103]],
 
          [[101,  82,  86],
           [ 84,  65,  69],
           [ 54,  32,  35],
           ...,
           [ 89,  79,  83],
           [163, 153, 157],
           [132, 122, 126]],
 
          [[ 78,  61,  62],
           [ 62,  45,  46],
           [ 63,  44,  46],
           ...,
           [ 77,  71,  76],
           [157, 151, 156],
           [121, 115, 120]],
 
          ...,
 
          [[102, 130,  55],
           [113, 141,  66],
           [115, 145,  67],
           ...,
           [119, 142,  80],
           [118, 141,  77],
           [ 95, 118,  54]],
 
          [[100, 128,  51],
           [114, 142,  65],
           [115, 145,  67],
           ...,
           [1